[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MouseLand/rastermap/blob/main/notebooks/rastermap_zebrafish.ipynb)

# Rastermap sorting of zebrafish neural activity

We will use a zebrafish wholebrain neural activity recording from [Chen*, Mu*, Hu*, Kuan* et al 2018](https://doi.org/10.1016/j.neuron.2018.09.042). The full dataset is available [here](https://doi.org/10.25378/janelia.7272617). The recordings were performed at a rate of 2.1 Hz. We took the neurons with the highest variance signals and deconvolved them to reduce long timescales in the data from the calcium sensor.

First we will install the required packages, if not already installed. If on google colab, it will require you to click the "RESTART RUNTIME" button because we are updating numpy.

In [ ]:
!pip install numpy>=1.24 # (required for google colab)
!pip install rastermap 
!pip install matplotlib

### Load data and import libraries

If not already downloaded, the following cell will automatically download the processed data stored [here](https://osf.io/2w8pa).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# importing rastermap
# (this will be slow the first time since it is compiling the numba functions)
from rastermap import Rastermap, utils
from scipy.stats import zscore

# download spontaneous activity
filename = utils.download_data(data_type="fish")

dat = np.load(filename)

# spks is neurons by time
# (each timepoint is 476 ms)
spks = dat["spks"]
n_neurons, n_time = spks.shape
print(f"{n_neurons} neurons by {n_time} timepoints")

# zscore activity (each neuron activity trace is then mean 0 and standard-deviation 1)
spks = zscore(spks, axis=1)

# XYZ position of each neuron in the recording
xyz = dat["xyz"]

# load the stimulus times
stims = dat["stims"]
# stim colors
fcolor = np.zeros((stims.max()+1, 4))
fcolor[0:3] = np.array([[0., 0.5, 1.0, 1.0], [1.0, 0.0, 1.0, 1.0], 
                        [1., 1., 1., 1.]])
fcolor[8:12] = np.array([[1.0,0.0,0,1],
                        [0.8,1.0,0,1], [0,0,1,1], [0,1,1,1]])

# load the fictive swimming
swimming = dat["swimming"]


### Run Rastermap

Let's sort the single neurons with Rastermap, with clustering and upsampling:

In [ ]:
model = Rastermap(n_clusters=100, # number of clusters to compute
                  n_PCs=200, # number of PCs to use
                  locality=0.1, # locality in sorting is low here to get more global sorting (this is a value from 0-1)
                  time_lag_window=5, # use future timepoints to compute correlation
                  grid_upsample=10, # default value, 10 is good for large recordings
                ).fit(spks)
y = model.embedding # neurons x 1
isort = model.isort

Let's create superneurons from Rastermap -- we sort the data and then sum over neighboring neurons:

In [ ]:
nbin = 50 # number of neurons to bin over 
sn = utils.bin1d(spks[isort], bin_size=nbin, axis=0) # bin over neuron axis

### Visualization

Use the Rastermap sorting to visualize the neural activity (see Figure 4 from the paper for the stimulus legend):

In [ ]:
# timepoints to visualize
xmin = 5700
xmax = 7860

# make figure with grid for easy plotting
fig = plt.figure(figsize=(12,6), dpi=200)
grid = plt.GridSpec(9, 20, figure=fig, wspace = 0.05, hspace = 0.3)

# plot swimming speed
ax = plt.subplot(grid[0, :-1])
ax.plot(swimming[xmin:xmax, 0], color=fcolor[11])
ax.plot(swimming[xmin:xmax, 1], color=fcolor[10])
ax.set_xlim([0, xmax-xmin])
ax.axis("off")
ax.set_title("swimming speed")

# plot superneuron activity
ax = plt.subplot(grid[1:, :-1])
ax.imshow(sn[:, xmin:xmax], cmap="gray_r", vmin=0, vmax=0.8, aspect="auto")
ax.set_xlabel("time")
ax.set_ylabel("superneurons")

# color time periods by stimuli
from matplotlib import patches
nn = sn.shape[0]
stims_t = stims[xmin:xmax]
starts = np.nonzero(np.diff(stims_t))
starts = np.append(np.array([0]), starts)
starts = np.append(starts, np.array([len(stims_t)-1]))
for n in range(len(starts)-1):
    start = starts[n]+1
    stype = stims_t[start]
    if stype!=3:
        width = starts[n+1] - start + min(0, start)
        start = max(0, start)
        ax.add_patch(
                patches.Rectangle(xy=(start, 0), width=width,
                            height=nn, facecolor=fcolor[stype], 
                            edgecolor=None, alpha=0.15*(stype!=2)))


ax = plt.subplot(grid[1:, -1])
ax.imshow(np.arange(0, len(sn))[:,np.newaxis], cmap="gist_ncar", aspect="auto")
ax.axis("off")

Color the neurons by their position in the rastermap:

In [ ]:
plt.figure(figsize=(5, 5))
plt.scatter(xyz[:,1], xyz[:,0], s=1, c=y, cmap="gist_ncar", alpha=0.25)
plt.xlabel('X position')
plt.ylabel('Y position')
plt.axis("square")

We can also divide the rastermap into sections to more easily visualize spatial relations (as in Figure 4):

In [ ]:
ny, nx = 3, 6
nxy = nx * ny

# divide into nxy sections
nb = len(isort) // nxy
colors = plt.get_cmap("gist_ncar")(np.linspace(0, 0.9, nxy))

# make figure with grid for easy plotting
fig = plt.figure(figsize=(12,6), dpi=200)

grid = plt.GridSpec(ny, nx, figure=fig, wspace = 0.25, hspace = 0.1)
for j in range(nx):
    for k in range(ny):
        ax = plt.subplot(grid[k,j])
        # plot all neurons
        subsample = 25
        ax.scatter(xyz[:,1][::subsample], xyz[:,0][::subsample], s=2, alpha=1, 
                    color=0.9*np.ones(3), rasterized=True)
        ip = j + k*nx
        ix = isort[ip*nb : (ip+1)*nb]
        subsample = 1
        ax.scatter(xyz[ix,1][::subsample], xyz[ix,0][::subsample],
                    s=0.5, alpha=0.3, color=colors[ip])
        ax.axis("off")
        ax.axis("square")
        ax.text(0.1,0,str(ip+1), transform=ax.transAxes, ha="right")

### Settings

You can see all the rastermap settings with `Rastermap?`

In [ ]:
Rastermap?

### Outputs

All the attributes assigned to the Rastermap `model` are listed with `Rastermap.fit?`

In [ ]:
Rastermap.fit?